In [30]:
# imports

import pandas as pd
import os
import requests
import time
import pprint
from urllib.parse import quote_plus  



### DFCI_2014_PES study

In [17]:
# get data
#TODO import as objects from harvester
## add parameter to skip first 4 lines in patient and study sample data

init_mut_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_mutations.txt', sep='\t')
init_study_meta = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/meta_study.txt', sep='\t')
init_patient_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_patient.txt', sep='\t', skiprows=4)
init_sample_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_sample.txt', sep='\t', skiprows=4)


In [18]:
# clean variant data

# subset for necessary columns
mut_df = init_mut_df.filter(['Hugo_Symbol',
                        'Chromosome',
                        'Start_Position', 
                        'End_Position',
                        'Consequence',
                        'Variant_Classification',
                        'Variant_Type',
                        'Reference_Allele',
                        'Tumor_Seq_Allele2',
                        'Tumor_Sample_Barcode',
                        'Sequence_Source',
                        'HGVSc',
                        'HGVSp',
                        'HGVSp_Short',
                        'Transcript_ID',
                        'RefSeq',
                        'Protein_position'
                       ], axis=1)


# Strip whitespace and retry
mut_df.columns = mut_df.columns.str.strip()
mut_df = mut_df.rename(columns={'Tumor_Sample_Barcode': 'SAMPLE_ID'})

# Check duplicate count
num_duplicates = mut_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# TODO check duplicates for sanity check
# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(mut_df[mut_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# save duplicate rows to file
    dupes = mut_df[mut_df.duplicated(keep=False)]
    mut_df.to_csv('mut_dupes.csv', index=False)
    

    
# remove duplicates, but keep first occurrence
    mut_df = mut_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {mut_df.shape}")
else:
    print("No duplicate rows found.")

    
# print(mut_df.columns)
# # print(mut_df.head())

Number of duplicate rows : 495

Duplicate rows (excluding first instance):
      Hugo_Symbol Chromosome  Start_Position  End_Position  \
720         ABCA1          9       107607765     107607765   
730         ABCA3         16         2338066       2338066   
783         ABCC9         12        21954066      21954066   
813         ACACA         17        35640173      35640173   
829          ACHE          7       100490251     100490251   
...           ...        ...             ...           ...   
15134       STAG2          X       123179197     123179197   
15155     ZDHHC15          X        74742823      74742824   
15191       HUWE1          X        53579734      53579734   
15216     SHROOM4          X        50350700      50350700   
15218        SOX3          X       139586714     139586714   

              Consequence Variant_Classification Variant_Type  \
720      missense_variant      Missense_Mutation          SNP   
730      missense_variant      Missense_Mutation  

In [19]:
# clean patient data
# print(patient_df.shape)
# print(patient_df.columns)

# subset data
# subset for necessary columns
patient_df = init_patient_df.filter(['PATIENT_ID',
                        'AGE',
                        'SEX', 
                        'ETHNICITY',
                        'Consequence'
                       ], axis=1)

print(patient_df.columns)

# Check duplicate count
num_duplicates = patient_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(patient_df[patient_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    patient_df = patient_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {patient_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'AGE', 'SEX', 'ETHNICITY'], dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


In [20]:
# clean sample data

# print(sample_df.shape)
# print(sample_df.columns)
# print(sample_df.head())


# subset data
# subset for necessary columns
sample_df = init_sample_df.filter(['PATIENT_ID',
                                     'SAMPLE_ID',
                                     'SAMPLE_CLASS',
                                     'ONCOTREE_CODE',
                                     'CANCER_TYPE',
                                     'CANCER_TYPE_DETAILED',
                                     'TMB_NONSYNONYMOUS'
                                    ], axis=1)


print(sample_df.columns)


# Check duplicate count
num_duplicates = sample_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(sample_df[sample_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    sample_df = sample_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {sample_df.shape}")
else:
    print("No duplicate rows found.")

Index(['PATIENT_ID', 'SAMPLE_ID', 'SAMPLE_CLASS', 'ONCOTREE_CODE',
       'CANCER_TYPE', 'CANCER_TYPE_DETAILED', 'TMB_NONSYNONYMOUS'],
      dtype='object')
Number of duplicate rows : 0
No duplicate rows found.


In [21]:
# combine dataframes
## TODO: redo so that all columns are available and that records with no samples will be there. "left outer join"? will get NAs.

init_combined_df = mut_df.merge(sample_df, on='SAMPLE_ID', how='left')

# print(mut_df.columns)
# print(mut_df.shape)

# print(sample_df.columns)
# print(sample_df.shape)

# print(init_combined_df.columns)
# print(init_combined_df.shape)

#add patient_df
combined_df = init_combined_df.merge(patient_df, on='PATIENT_ID', how='left')

# print(patient_df.columns)
# print(patient_df.shape)

# print(combined_df.columns)
# print(combined_df.shape)


In [22]:

# add column for study id
study_id = init_study_meta.iloc[0, 0]
study_id = study_id.replace('cancer_study_identifier: ', '')
# study_id
combined_df['STUDY_ID'] = study_id


In [23]:


# Check duplicate count
num_duplicates = combined_df.duplicated().sum()
print(f"Number of duplicate rows : {num_duplicates}")

# print duplicates (excluding first instance)
if num_duplicates > 0:
    print("\nDuplicate rows (excluding first instance):")
    print(combined_df[combined_df.duplicated()])

# # print full duplicate groups (including the first occurrences)
#     print("\nAll rows involved in duplication:")
#     print(mut_df[mut_df.duplicated(keep=False)])

# remove duplicates, but keep first occurrence
    combined_df = combined_df.drop_duplicates()
    print(f"\nDataFrame shape after removing duplicates: {combined_df.shape}")
else:
    print("No duplicate rows found.")



Number of duplicate rows : 0
No duplicate rows found.


In [24]:

# remove cell lines

original_shape = combined_df.shape
print(f"Original shape: {original_shape}")

#lines to remove
removed_df = combined_df[combined_df['SAMPLE_CLASS'] == 'Cell line']

# remove cell lines
filtered_df = combined_df[combined_df['SAMPLE_CLASS'] != 'Cell line']

# calculate how many rows were removed
rows_removed = original_shape[0] - filtered_df.shape[0]
print(f"Removed {rows_removed} rows where SAMPLE_CLASS == 'Cell line'")

# print new shape
print(f"New shape: {filtered_df.shape}")

# reassign df
combined_df = filtered_df

removed_df.to_csv('cell_lines_removed.csv', index=False)
removed_df.value_counts("SAMPLE_CLASS")


Original shape: (14737, 27)
Removed 3123 rows where SAMPLE_CLASS == 'Cell line'
New shape: (11614, 27)


SAMPLE_CLASS
Cell line    3123
Name: count, dtype: int64

In [25]:
# combined_df.isna().sum()

# Hugo_Symbol                   0
# Chromosome                    0
# Start_Position                0
# End_Position                  0
# Consequence                 124 - some of the Variant_Classification=Silent have no consequence described
# Variant_Classification        0
# Variant_Type                  0
# Reference_Allele              0
# Tumor_Seq_Allele2             0
# SAMPLE_ID                     0
# Sequence_Source               0
# HGVSc                       136 - some of the Variant_Classification=Silent and all of the Variant_Classification=3'Flank and 5'Flank
# HGVSp                       346 
# Transcript_ID               124 - some of the Variant_Classification=Silent have no Transcript_ID described
# RefSeq                     1338
# Protein_position            334
# Gnomad_Notation               0
# PATIENT_ID                    0
# SAMPLE_CLASS                  0
# ONCOTREE_CODE                 0
# CANCER_TYPE                   0
# CANCER_TYPE_DETAILED          0
# TMB_NONSYNONYMOUS             0
# AGE                        4745 - some ages undisclosed
# SEX                           0
# ETHNICITY                 10900 - many patients' ethnicities undisclosed
# STUDY_ID                      0


# define output folder (will create it if needed)
output_dir = "value_counts_by_column"
os.makedirs(output_dir, exist_ok=True)

# loop through each column
for col in combined_df.columns:
    # build filename
    filename = f"value_counts_{col}.txt"
    filepath = os.path.join(output_dir, filename)
    
    # write counts to file
    with open(filepath, "w") as f:
        f.write(f"Value counts for column: {col}\n\n")
        f.write(combined_df[col].value_counts(dropna=False).to_string())
        f.write("\n")

print(f"✅ Value counts written for {len(combined_df.columns)} columns to folder: {output_dir}")


✅ Value counts written for 27 columns to folder: value_counts_by_column


In [26]:
#filling in NaNs - AGE, ETHNICITY, Consequence
# TODO add consequence with annotation

cols_to_fill = ['Consequence', 'AGE', 'ETHNICITY']
fill_value = "No_Data"

for col in cols_to_fill:
    combined_df[col] = combined_df[col].fillna(fill_value)

combined_df.isna().sum()

Hugo_Symbol                  0
Chromosome                   0
Start_Position               0
End_Position                 0
Consequence                  0
Variant_Classification       0
Variant_Type                 0
Reference_Allele             0
Tumor_Seq_Allele2            0
SAMPLE_ID                    0
Sequence_Source              0
HGVSc                      136
HGVSp                      346
HGVSp_Short                136
Transcript_ID              124
RefSeq                    1338
Protein_position           334
PATIENT_ID                   0
SAMPLE_CLASS                 0
ONCOTREE_CODE                0
CANCER_TYPE                  0
CANCER_TYPE_DETAILED         0
TMB_NONSYNONYMOUS            0
AGE                          0
SEX                          0
ETHNICITY                    0
STUDY_ID                     0
dtype: int64

In [ ]:
# correcting chromosome 23 to chromosome X or Y

# TODO. look at gene and see whether it maps to x or y for males
combined_df.to_csv('output0.csv', index=False)

#1 - try these examples in GUI
# 23-153175777-G-A
# 23-134494503-C-T
# 23-153223671-C-T
## yes these work

#2 - try this exampe programmatically
# 23-153175777-G-A  -----   ARHGAP4

# gnomAD  = "X-153175777-G-A"
# assembly = "GRCh37"                   # or "GRCh38"
# https:/normalize.cancervariants.org/variation/normalize?q=x-153223671-C-T
BASE_URL = "https://normalize.cancervariants.org/variation/"
HEADERS = {"Accept": "application/json"}


def test_tokenization(gnomAD):
    """Fetch gene from VICC variation normalizer"""
    url = f"{BASE_URL}normalize?q={gnomAD}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None



def chr23_females(df) -> pd.DataFrame:
    """
    Convert Chromosome 23 to 'X' for rows where SEX is female.
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns 'Chromosome' and 'SEX'.
    
    Returns
    -------
    dataframe
        The same DataFrame (modified in place) so you can chain if desired.
    """
    # Ensure we’re comparing like with like
    chr_col = df["Chromosome"].astype(str).str.strip()
    sex_col = df["SEX"].astype(str).str.upper().str.strip()   # handles 'F', 'f', 'Female', etc.
    
    mask = (chr_col == "23") & (sex_col.str.startswith("F"))
    df.loc[mask, "Chromosome"] = "X"
    return df

combined_df = chr23_females(combined_df)
combined_df.to_csv('output_post_Female_X.csv', index=False)

variant = "23-153175777-G-A"
PATTERN = re.compile(r'^23-')        # anchored ^ so only the chromosome prefix is substituted
gene = "ARHGAP4"

def chr23_to_X(variant: str) -> str:
    """Return `variant` with leading '23-' swapped to 'X-'; otherwise unchanged."""
    return PATTERN.sub('X-', variant)
def chr23_to_Y(variant: str) _> str:
    """Return `variant` with leading '23-' swapped to 'Y-'; otherwise unchanged."""
    return PATTERN.sub('Y-', variant)


def chr_23_male_det(variant):
    # change to X if position > 60 000 000
    pos = int(variant.split('-')[1]) 
    print(f"Variant is at position {pos}")
    if pos > 60000000:
        print(f"Variant {variant} is located at a position beyond the length of the Y chromosome.")
        # df["Variant"] = df["Variant"].str.replace(r'^23-', 'X-', regex=True)
        new_notation = chr23_to_X(variant)
        print(f"New variant notation: {new_notation}")
    elif pos < 60000000:
        print("Determining if variant is on X or Y...")
        Y_variant = chr23_to_Y(variant)
        print(f"testing {Y_variant} for tokenization")
        r = test_tokenization(Y_variant)
        warnings = r['warnings']
        if warnings == "[]":
            yes_Y_chrom = Y_variant
            print(f"{variant} is present on the Y chromosome")
        else:
            print(f"{variant} is NOT present on the Y chromosome")
        X_variant = chr23_to_X(variant)
        print(f"testing {X_variant} for tokenization") 
        r = test_tokenization(X_variant)
        warnings = r['warnings']
        if warnings != "[]":
            yes_X_chrom = X_variant
            print(f"{variant} is present on the X chromosome")
        else:
            print(f"{variant} is NOT present on the X chromosome")


        
        
        print Y_variant

    






# def chr_23_males(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Where SEX is Male and Chromosome is 23, apply a series of checks:

#     • If Start_Position > 60,000,000  → set Chromosome to 'X'.
#     • Otherwise build gnomAD notation twice (Y-chromosome and X-chromosome),
#       run `test_tokenization()` on each, and:
#         – if only X succeeds → Chromosome = 'X'
#         – if only Y succeeds → Chromosome = 'Y'
#         – if both or neither succeed → leave as 23 and flag the row.

#     Returns
#     -------
#     pd.DataFrame
#         The same DataFrame (modified in place).
#     """
#     # normalize columns
#     chr_col = df["Chromosome"].astype(str).str.strip()
#     sex_col = df["SEX"].astype(str).str.upper().str.strip()
#     pos_col = df["Start_Position"]

#     # rows that are chr 23 **and** male
#     mask_male_23 = (chr_col == "23") & sex_col.str.startswith("M")

#     # STEP 1
#     # change to X if position > 60 000 000
#     mask_high_pos = mask_male_23 & (pos_col > 60_000_000)
#     df.loc[mask_high_pos, "Chromosome"] = "X"

#     # STEP 2
#     # gnomAD strings and tokenizer checks for the remaining rows
#     remaining = mask_male_23 & ~mask_high_pos

#     if remaining.any():                         # skip if nothing left
#         gnomad_Y = df.loc[remaining].apply(
#             lambda r: f"Y-{r.Start_Position}-{r.Reference_Allele}-{r.Tumor_Seq_Allele2}",
#             axis=1
#         )
#         gnomad_X = df.loc[remaining].apply(
#             lambda r: f"X-{r.Start_Position}-{r.Reference_Allele}-{r.Tumor_Seq_Allele2}",
#             axis=1
#         )

#         ok_Y = test_tokenization(gnomad_Y)
#         ok_X = test_tokenization(gnomad_X)
#         print(ok_Y)
#         print(ok_X)

#         # rows where only one succeeds
#         only_X = remaining & ok_X & ~ok_Y
#         only_Y = remaining & ok_Y & ~ok_X

#         df.loc[only_X, "Chromosome"] = "X"
#         df.loc[only_Y, "Chromosome"] = "Y"

#         # rows where result is ambiguous → flag
#         ambiguous = remaining & ~(only_X | only_Y)
#         df.loc[ambiguous, "XY_flagged"] = True

#     return df


chr23_females(combined_df)
chr_23_males (combined_df)




# combined_df = chr23_females(combined_df)
# position 1428413



# r = test_tokenization("X-153175777-G-A")
# r








# # write to file
combined_df.to_csv('output2.csv', index=False)

# print("hello")

SyntaxError: expected ':' (2580060520.py, line 66)

In [ ]:
# construct Gnomad variant ID column
combined_df["Gnomad_Notation"] = combined_df.apply(
    lambda row: f"{row['Chromosome']}-{row['Start_Position']}-{row['Reference_Allele']}-{row['Tumor_Seq_Allele2']}",
    axis=1
)

In [ ]:
# remove variant dupes per patient

# find duplicated (PATIENT_ID, Gnomad_Notation) pairs
dupe_mask = combined_df.duplicated(subset=["PATIENT_ID", "Gnomad_Notation"], keep="first")
# new DataFrame with the duplicated rows
patient_variant_dupes = combined_df[dupe_mask]
# remove those rows from the original DataFrame
combined_df_cleaned = combined_df[~dupe_mask]
# write removed rows to file
patient_variant_dupes.to_csv("patient_variant_dupes.csv", index=False)
# print the number of rows removed
print(f"Removed {patient_variant_dupes.shape[0]} rows with duplicated Gnomad_Notation per PATIENT_ID.")
# reassign dataframe:
combined_df = combined_df_cleaned

Removed 134 rows with duplicated Gnomad_Notation per PATIENT_ID.
